In [15]:
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations (normalization and conversion to tensors)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the MNIST dataset for both training and testing, move data to GPU
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

class SmallNet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# Move model to GPU
model = SmallNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Training loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # Move data to GPU
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # Move data to GPU
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Train and test the model
for epoch in range(1, 11):  # You can adjust the number of epochs as needed
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303029
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.225705
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.089896
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.081704
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.050931
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.051239
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.168388
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.066811
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.021731
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.011314

Test set: Average loss: 0.0007, Accuracy: 98.52%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.120834
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.059694
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.063997
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.026672
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.024863
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.084564
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.034243
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.062646
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.012279
Train Epoch:

In [ ]:
class Resnet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

In [29]:
class CustomFcLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(CustomFcLayer, self).__init__()
        self.fc = nn.Linear(in_features, out_features)

    def forward(self, x):
        x = self.fc(x)
        return x

# Load the pre-trained ResNet-18 model and modify it for MNIST
resnet18 = models.resnet18(pretrained=True)
num_features = resnet18.fc.in_features
resnet18.fc = CustomFcLayer(num_features, 10)  # 10 classes for MNIST

# Move the model to GPU
model = resnet18.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # Move data to GPU
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Test loop
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # Move data to GPU
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Train and test the model
for epoch in range(1, 11):  # You can adjust the number of epochs as needed
    train(epoch)
    test()

NameError: name 'models' is not defined